# Time travel
Using the rank cluster, we can run our collection of real search terms against the new mapping with the new query structure. We can then analyse the results according to the same set of metrics as we used for the data which was collected in real time. In other words, we can look at how search _would have_ performed if we had made these changes earlier. It's a time-travelling A/B test.